<a href="https://colab.research.google.com/github/NehaSontakk/RNA-sequencing-fastq-to-differential-gene-expression/blob/main/RPKM%2C_TPM%2C_FPKM%2C_CPM_calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculate RPKM/FPKM and TPM Normalizations

Creating a directory to store your coverage files

In [2]:
!mkdir /content/data/

mkdir: cannot create directory ‘/content/data/’: File exists


In [3]:
import pandas as pd
from glob import glob

### Calculate RPKM(single end) /FPKM (paired end)

In [4]:
def rpkm_calculation(df):
  #normalize for read depth
  total_reads = df['Mapped_Read_Count'].sum()
  #per million scaling
  permil_norm = total_reads/1000000
  df['RPM_Norm'] = df['Mapped_Read_Count']/permil_norm
  #normalize for gene length in kilobase
  df['RPKM/FPKM'] = df['RPM_Norm']/(df['Gene_Length']/1000)
  return df.drop(columns='RPM_Norm')

### Calculate TPM

This is like RPKM/FPKM only the order of operations is switched

In [5]:
def tpm_calculation(df):
  #normalize for gene length
  df['K_Norm'] = df['Mapped_Read_Count']/(df['Gene_Length']/1000)
  #normalize for read depth
  total_reads = df['K_Norm'].sum()
  #per million scaling
  permil_norm = total_reads/1000000
  df['TPM'] = df['K_Norm']/permil_norm
  return df.drop(columns='K_Norm')

### Combine

In [6]:
def tpm_rpkm(loc):
  df = pd.read_table(loc,sep="\t",header=None,names=["NC_Name","Start","End","Gene_Name","Gene","Strand","Mapped_Read_Count","Length_of_gene_covered_by_mapped_reads","Gene_Length","Coverage"])
  df_rpkm = rpkm_calculation(df)
  df_rpkm_tpm = tpm_calculation(df_rpkm)
  return df_rpkm_tpm

### Final Data Sets

In [7]:
!mkdir /content/RPKM_FPKM_TPM_Outputs/

In [8]:
for i in glob("/content/data/*.cov"):
  name = i.split("/")[-1].split(".cov")[0]
  tpm_rpkm(i).to_excel("/content/RPKM_FPKM_TPM_Outputs/"+name+".xlsx")

In [9]:
import os
os.system("zip -r /content/RPKM_FPKM_TPM_Outputs.zip /content/RPKM_FPKM_TPM_Outputs/")

0